### **HuggingFace Login**

In [10]:
from huggingface_hub import login
login()

### **Import Libairies**

In [29]:
from IPython.display import display

import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, f1_score
from datasets import Dataset


### **Read Datasets**

In [30]:
'''
df_train = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/train.csv')
df_test = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/test.csv')
'''
df_train = pd.read_csv('C:/Users/cauchepy/Datasets/ComputerVisionImages/kaggle_digitrecognizer/train.csv')
df_test = pd.read_csv('C:/Users/cauchepy/Datasets/ComputerVisionImages/kaggle_digitrecognizer/test.csv')

### **Short Analysis**

In [31]:
distinct_labels = df_train['label'].value_counts()
display(pd.DataFrame(distinct_labels))
classes = len(df_train["label"].unique())

,count
label,
1,4684
7,4401
3,4351
9,4188
2,4177
6,4137
0,4132
4,4072
8,4063


### **Preprocess Datasets**

In [32]:
y_train = np.array(df_train['label'])
X_train = np.array(df_train.drop('label', axis=1))
X_test = np.array(df_test)

In [33]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

### **Model Assessment**

In [35]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=df_train["label"].unique(), index=df_train["label"].unique())
    
    return metrics_df, cm_df

In [41]:
X_train = X_train[:100]
y_train = y_train[:100]
X_valid = X_valid[:100]
y_valid = y_valid[:100]

### **Transformers VIT**

In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor, AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

def preprocess_images(images):
    # Convert to PIL Images
    pil_images = [Image.fromarray((img * 255).astype(np.uint8)).convert("RGB") if isinstance(img, np.ndarray) else img.convert("RGB") for img in images]
    processed_inputs = processor(images=pil_images, return_tensors="pt")['pixel_values']
    return processed_inputs

X_train_tensor = preprocess_images(X_train)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_valid_tensor = preprocess_images(X_valid)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)


# Initialize ViT model
vit_classifier = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k', 
    num_labels=10
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_classifier.to(device)

# Training setup
optimizer = AdamW(vit_classifier.parameters(), lr=1e-5)
epochs = 3

# Training loop
for epoch in tqdm(range(epochs)):
    vit_classifier.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = vit_classifier(inputs, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

# Evaluation on validation set
vit_classifier.eval()
all_vit_classifier_preds, all_vit_classifier_labels = [], []

with torch.no_grad():
    for batch in valid_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        
        all_vit_classifier_preds.extend(preds.cpu().numpy())
        all_vit_classifier_labels.extend(labels.cpu().numpy())

# Calculate accuracy and classification report
accuracy = accuracy_score(all_vit_classifier_labels, all_vit_classifier_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(all_vit_classifier_labels, all_vit_classifier_preds))

# Additional assessment (if function `evaluate_classifier` is defined)
vit_classifier_assessment = evaluate_classifier(all_vit_classifier_labels, all_vit_classifier_preds)
print(vit_classifier_assessment)

vit_classifier.push_to_hub("yanncauchepin/kaggle_digitrecognizer_vit_model")

100%|██████████| 100/100 [00:00<00:00, 294.21it/s]
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\cauchepy\Code\.venv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  3%|▎         | 1/30 [00:19<09:35, 19.85s/it]

Epoch 1/30, Training Loss: 2.3238


  7%|▋         | 2/30 [00:49<12:02, 25.80s/it]

Epoch 2/30, Training Loss: 2.2958


 10%|█         | 3/30 [01:27<14:05, 31.30s/it]

...

Epoch 12/30, Training Loss: 2.1192


 43%|████▎     | 13/30 [08:36<12:19, 43.48s/it]

Epoch 13/30, Training Loss: 2.0977


 47%|████▋     | 14/30 [09:23<11:49, 44.32s/it]

Epoch 14/30, Training Loss: 2.0695


In [ ]:
X_test_tensor = preprocess_images(X_test)
test_dataset = TensorDataset(X_test_tensor)

test_loader = DataLoader(test_dataset, batch_size=3)

vit_classifier.eval()
test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = batch.to(device)  
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        test_preds.extend(preds.cpu().numpy())

vit_classifier_submission_df = pd.DataFrame({
    "ImageId": range(1, len(test_preds) + 1),
    "Label": test_preds
})
from datasets import Dataset
Dataset.from_pandas(vit_classifier_submission_df).push_to_hub("yanncauchepin/kaggle_digitrecognizer_vit_submission_df")


  0%|          | 0/9334 [00:00<?, ?it/s]/tmp/ipykernel_25585/3633612418.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(self.files[idx])
100%|████████

AttributeError: type object 'Dataset' has no attribute 'from_pandas'